### Projet 1 - Contrôle de la propagation d'une maladie contagieuse

**À rendre avant le 19.12.2024 minuit.**

---

#### Synopsis

Dans ce projet, vous êtes confrontés à un problème de santé publique simulé : contrôler la propagation d'une maladie contagieuse dans une population humaine. Des experts en épidémiologie ont collecté des données en mesurant cinq caractéristiques distinctes (_x1_ à _x5_) pour déterminer si une personne est affectée par la maladie (y=1). Les résultats de ces mesures sont disponibles dans le fichier _[./projet_1_data.csv](./projet_1_data.csv)_.

L'objectif est de concevoir et comparer des modèles capables de déterminer si une personne est infectée ou non, en utilisant des données fictives basées sur cinq variables (_x1_ à _x5_). Cependant, nous ne savons pas quelles variables sont réellement pertinentes pour prédire l'infection.

Votre mission est de trouver les variables les plus importantes pour réduire au minimum les données invasives à collecter, tout en garantissant que les faux négatifs (cas de maladie non détectés) ne dépassent pas 10% de la population affectée. Ce seuil est essentiel pour limiter la propagation de la maladie. Simultanément, vous devez minimiser les faux positifs pour réduire les coûts des mesures de contrôle supplémentaires, comme des tests de confirmation.

---

**Important** : Ce problème est une simulation avec des variables fictives. Il ne repose pas sur des connaissances médicales réelles. Ne vous appuyez pas sur une expertise métier pour résoudre ce problème : basez vos décisions sur les données et les outils d'analyse.

---

#### Objectifs

1. **Construire les modèles** :
   - Identifier les interactions entre les variables explicatives et la variable cible **y**.
   - Développez et comparez **deux** modèles de machine learning **de type différents** en utilisant `scikit-learn`.

3. **Sélection des variables importantes** :
    -  Dans la construction des modèles, déterminez les variables les plus pertinentes (parmis _x1_ à _x5_) pour la prédiction de l'infection, en justifiant vos choix.

5. **Évaluer les performances de vos modèles** :
   - Utiliser des métriques adaptées.
   - Visualiser la qualité des prédictions sur une partie des données mises à votre disposition.

---
#### Livrables

1. **Modèles** :
   - **deux** modèles de prédiction de type différent, avec seuil de detection pour chaque.

2. **Caratèristiques** :
   - Liste(s) des caractéristiques pertinentes.

3. **Rapport** :
   - Fournissez un _[rapport](rapport.ipynb)_ bref et clair expliquant vos choix, vos résultats et les performances des modèles. Assurez-vous de valider vos résultats avec des métriques appropriées.

---

#### Détails

1. **Réduction des variables** :
   - Identifiez les variables clés parmi _x1_ à _x5_ pour minimiser la collecte de données inutiles. Justifiez votre approche, que ce soit par analyse exploratoire, importance des caractéristiques ou autre méthode.

2. **Construction des modèles** :
   - Concevez deux modèles prédictifs différents avec `scikit-learn` pour prédire si une personne est infectée ou non.

3. **Validation des performances** :
   - Évaluez vos modèles en utilisant des métriques appropriées. Utilisez ces résultats pour comparer les deux modèles.

4. **Ajustement du seuil** :
   - Modifiez le seuil de détection pour limiter les faux négatifs à un maximum de 10% (pour contenir la propagation), tout en minimisant les faux positifs pour réduire les coûts des contrôles inutiles.

5. **Outils suggérés** :
   - Utilisez `numpy`, `pandas` et `scikit-learn` pour votre analyse et vos modèles.

6. **Nettoyage avant soumission** :
   - Avant de soumettre votre travail, assurez-vous que votre notebook s'exécute correctement de bout en bout en utilisant l'option _Restart Kernel and Run All Cells..._. Ensuite exécutez _Restart Kernel and Clear All Outputs..._ pour sauvegarder un notebook propre.

---

#### Evaluation

Cette question admet plusieurs solutions possibles. Une partie de la note sera attribuée en fonction de la cohérence et de la justification de vos choix. Votre rapport devra inclure une discussion expliquant les méthodes utilisées, ainsi que des résultats obtenus.

Pour maximiser votre impact, comparez les performances des deux modèles que vous avez développés et justifiez le choix de celui que vous considérez comme optimal.

---

#### Barème - sur 6 points

1. **Qualité de la méthode et du code** :
    - Noté sur 3 points, en tenant compte de la clarté, de l'organisation et de l'efficacité des approches utilisées.

2. **Vérification et analyse des résultats** :
    - Noté sur 3 points, en fonction de la rigueur dans l'évaluation des modèles et de la pertinence des conclusions.

3. **Fonctionnalité** :
    - Des points seront déduits si le notebook n'est pas fonctionnel, en fonction de la gravité des problèmes rencontrés et du nombre de corrections nécessaires pour le rendre opérationnel.


Les notes incluent une évaluation des réponses fournies dans le rapport, en particulier la justification des choix effectués et l'interprétation des résultats et performances des modèles.


---


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Install seaborn if not already installed
import subprocess
import sys

try:
    import seaborn as sns
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "seaborn"])
    import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, accuracy_score


# Projet 1 - Contrôle de la propagation d'une maladie contagieuse

## Contexte et Objectifs

Vous disposez de données simulées sur une population humaine, avec cinq variables explicatives (x1 à x5) et une variable cible (y) indiquant si une personne est infectée (y=1) ou non (y=0).

**Objectifs** :
1. Identifier les variables les plus importantes pour prédire l'infection.
2. Construire et comparer deux modèles de machine learning différents (ex : Régression Logistique et Forêt Aléatoire).
3. Ajuster le seuil de prédiction pour garantir que les faux négatifs ne dépassent pas 10% des individus infectés, tout en minimisant les faux positifs.
4. Réduire idéalement le nombre de variables collectées sans dégrader fortement la capacité du modèle à détecter l'infection.

Nous utiliserons `scikit-learn`, `pandas`, `numpy`, `matplotlib` et `seaborn`.

---

### Chargement des données

Le fichier `projet_1_data.csv` contient les colonnes suivantes :
- x1, x2, x3, x4, x5 : variables explicatives
- y : variable cible (1 si infecté, 0 sinon)

On charge le jeu de données et on en fait un aperçu.

In [ ]:
data = pd.read_csv("projet_1_data.csv")
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
data = pd.read_csv("projet_1_data.csv")

# Quick look at the data
print(data.head())
print(data.info())
print(data.describe())

# Distribution of the target variable
print("Répartition de la variable cible (y) :")
print(data['y'].value_counts(normalize=True))

# Pairplot to visualize relationships and distributions
# 'hue' is set to 'y' to color the points by the target class
sns.pairplot(data, hue='y', diag_kind='hist', vars=['x1','x2','x3','x4','x5'])
plt.suptitle("Visualisation initiale des relations entre variables (coloration par y)", y=1.02)
plt.show()

# Correlation heatmap
plt.figure(figsize=(8,6))
sns.heatmap(data.corr(), annot=True, cmap="coolwarm")
plt.title("Matrice de corrélation entre les variables")
plt.show()

### Analyse exploratoire

- Vérifions la distribution de la variable cible.
- Examinons la corrélation entre les variables pour avoir une première idée des relations.

In [ ]:
print("Répartition de la variable cible (y) :")
print(data['y'].value_counts(normalize=True))

plt.figure(figsize=(8,6))
sns.heatmap(data.corr(), annot=True, cmap="coolwarm")
plt.title("Matrice de corrélation entre les variables")
plt.show()

**Observations préliminaires :**

- La variable cible `y` est déséquilibrée (environ 20% d'infectés et 80% de non-infectés).
- Les corrélations entre `x1`, `x2`, `x3`, `x4`, `x5` et `y` semblent faibles, mais une faible corrélation linéaire ne signifie pas forcément une absence de relation prédictive.
  
Nous compterons sur les modèles pour estimer l'importance des variables.

---

### Séparation des données

Avant toute modélisation, on sépare les données en jeu d'entraînement (train) et de test.  
Le jeu d'entraînement sert à ajuster les modèles, et le jeu de test évalue la performance sur des données non vues afin de détecter un éventuel surapprentissage.

In [7]:
X = data[['x1','x2','x3','x4','x5']]
y = data['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Sélection des variables importantes (estimation)

On va entraîner rapidement une Forêt Aléatoire sur toutes les variables pour évaluer leur importance.  
Cela nous donnera une indication sur les variables potentiellement essentielles.

In [ ]:
rf_temp = RandomForestClassifier(random_state=42)
rf_temp.fit(X_train, y_train)
importances = rf_temp.feature_importances_

feature_importances = pd.DataFrame({'feature': X.columns, 'importance': importances})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

plt.figure(figsize=(6,4))
sns.barplot(x='importance', y='feature', data=feature_importances, orient='h')
plt.title("Importance des variables - Forêt Aléatoire (approche rapide)")
plt.show()

feature_importances

D'après ce premier aperçu, certaines variables (par exemple `x2` et `x1`) semblent avoir plus d'importance. Nous conserverons cette information pour éventuellement réduire le nombre de variables par la suite.

---

### Construction des modèles

Nous allons construire deux modèles différents pour comparer leurs performances :

1. **Régression Logistique**
2. **Forêt Aléatoire**

Ensuite, nous évaluerons leurs performances sur le jeu de test.

In [9]:
# Modèle 1 : Régression Logistique
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

y_pred_logreg = logreg.predict(X_test)
y_proba_logreg = logreg.predict_proba(X_test)[:,1]

# Modèle 2 : Forêt Aléatoire
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:,1]

### Évaluation des performances initiales

Nous utiliserons :
- Matrice de confusion
- Accuracy, Precision, Recall, F1-score
- AUC (ROC)

Ces métriques seront comparées sur le jeu de test.

In [ ]:
def print_metrics(model_name, y_true, y_pred, y_proba):
    print(f"=== {model_name} ===")
    cm = confusion_matrix(y_true, y_pred)
    print("Matrice de confusion :\n", cm)
    print("\nRapport de classification :\n", classification_report(y_true, y_pred))
    auc = roc_auc_score(y_true, y_proba)
    print("AUC-ROC :", auc)
    print("---------")

print_metrics("Régression Logistique", y_test, y_pred_logreg, y_proba_logreg)
print_metrics("Forêt Aléatoire", y_test, y_pred_rf, y_proba_rf)

### Comparaison des modèles via la courbe ROC

La courbe ROC permet de visualiser le compromis entre Taux de Vrais Positifs (TPR ou Recall) et Taux de Faux Positifs (FPR).

In [ ]:
#AUC = 1.0 (Overfitting is evident, but not necessarily problematic if generalization is still good.)
fpr_log, tpr_log, _ = roc_curve(y_test, y_proba_logreg)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_proba_rf)

plt.figure(figsize=(8,6))
plt.plot(fpr_log, tpr_log, label=f"Logistic Regression (AUC={roc_auc_score(y_test, y_proba_logreg):.2f})")
plt.plot(fpr_rf, tpr_rf, label=f"Random Forest (AUC={roc_auc_score(y_test, y_proba_rf):.2f})")
plt.plot([0,1],[0,1],'--',color='gray')
plt.xlabel("Taux de Faux Positifs (FPR)")
plt.ylabel("Taux de Vrais Positifs (TPR)")
plt.title("Courbes ROC")
plt.legend()
plt.show()

**Analyse** :

- La Régression Logistique a une AUC proche de 0.48 (environ aléatoire) et ne détecte presque aucun infecté (recall sur la classe 1 très faible).
- La Forêt Aléatoire a une AUC d’environ 0.95 sur le jeu de test, ce qui est excellent. Elle identifie beaucoup plus d'individus infectés.

Clairement, la Forêt Aléatoire est plus performante sur ce dataset.

---

### Ajustement du seuil

Objectif : Les faux négatifs (personnes infectées non détectées) doivent représenter au maximum 10% de tous les infectés.  
C’est-à-dire : FNR = FN / (FN + TP) ≤ 0.10, donc Sensibilité (TPR) ≥ 0.90.

Nous allons ajuster le seuil de décision (par défaut 0.5) sur les prédictions probabilistes de la Forêt Aléatoire.

In [ ]:
thresholds = np.linspace(0,1,101)
valid_thresholds = []

for t in thresholds:
    y_pred_t = (y_proba_rf >= t).astype(int)
    cm_t = confusion_matrix(y_test, y_pred_t)
    TN, FP, FN, TP = cm_t.ravel()
    sens = TP / (TP + FN) if (TP+FN)>0 else 0
    spec = TN / (TN + FP) if (TN+FP)>0 else 0
    if sens >= 0.90:
        # On enregistre les seuils qui atteignent au moins 90% de sensibilité
        valid_thresholds.append((t, sens, spec))

if len(valid_thresholds) > 0:
    # On choisit le seuil avec la meilleure spécificité parmi ceux qui ont sensibilité ≥ 0.90
    chosen_threshold, chosen_sens, chosen_spec = max(valid_thresholds, key=lambda x: x[2])
else:
    # Si aucun seuil n'atteint 90% de sensibilité, on reste à 0.5 (peu probable vu la perf de la RF)
    chosen_threshold, chosen_sens, chosen_spec = 0.5, None, None

chosen_threshold, chosen_sens, chosen_spec

In [ ]:
y_pred_adj = (y_proba_rf >= chosen_threshold).astype(int)
cm_adj = confusion_matrix(y_test, y_pred_adj)
TN, FP, FN, TP = cm_adj.ravel()

print("Matrice de confusion avec seuil ajusté:\n", cm_adj)
print("\nSensibilité (TPR):", TP/(TP+FN))
print("Spécificité (TNR):", TN/(TN+FP))
print("Taux de Faux Négatifs (FNR):", FN/(FN+TP))
print("Taux de Faux Positifs (FPR):", FP/(FP+TN))

**Analyse** :  
- Nous avons maintenant une sensibilité ≥ 90%, donc moins de 10% de faux négatifs parmi les infectés.
- Cela pourrait augmenter les faux positifs, mais l'idée est de mieux contrôler la propagation de la maladie en ne ratant presque aucun cas infecté.

---

### Réduction du nombre de variables

Testons un modèle Forêt Aléatoire avec uniquement les variables les plus importantes (par exemple `x2` et `x1` identifiées plus haut).

Puis réajustons le seuil de la même manière et comparons.

In [ ]:
X_train_reduced = X_train[['x2','x1']]
X_test_reduced = X_test[['x2','x1']]

rf_reduced = RandomForestClassifier(random_state=42)
rf_reduced.fit(X_train_reduced, y_train)

y_proba_rf_reduced = rf_reduced.predict_proba(X_test_reduced)[:,1]

thresholds = np.linspace(0,1,101)
valid_thresholds = []
for t in thresholds:
    y_pred_t = (y_proba_rf_reduced >= t).astype(int)
    cm_t = confusion_matrix(y_test, y_pred_t)
    TN_r, FP_r, FN_r, TP_r = cm_t.ravel()
    sens = TP_r / (TP_r + FN_r) if (TP_r+FN_r)>0 else 0
    spec = TN_r / (TN_r + FP_r) if (TN_r+FP_r)>0 else 0
    if sens >= 0.90:
        valid_thresholds.append((t, sens, spec))

if len(valid_thresholds) > 0:
    chosen_threshold_reduced, chosen_sens_reduced, chosen_spec_reduced = max(valid_thresholds, key=lambda x: x[2])
else:
    chosen_threshold_reduced, chosen_sens_reduced, chosen_spec_reduced = 0.5, None, None

y_pred_adj_reduced = (y_proba_rf_reduced >= chosen_threshold_reduced).astype(int)
cm_adj_reduced = confusion_matrix(y_test, y_pred_adj_reduced)
TN_r, FP_r, FN_r, TP_r = cm_adj_reduced.ravel()

print("Matrice de confusion (réduit):\n", cm_adj_reduced)
print("\nSensibilité (TPR):", TP_r/(TP_r+FN_r))
print("Taux de Faux Négatifs (FNR):", FN_r/(FN_r+TP_r))
print("Taux de Faux Positifs (FPR):", FP_r/(FP_r+TN_r))

**Analyse** :  
- En réduisant à x1 et x2, nous pourrions conserver une sensibilité élevée (≥90%) après ajustement du seuil, mais le taux de faux positifs augmente peut-être.
- Ce compromis doit être jugé selon les contraintes du problème. Réduire les données collectées diminue les coûts et l'invasion de la vie privée, mais augmente les tests inutiles.
- Selon la priorité, on peut soit conserver plus de variables pour réduire les faux positifs, soit se contenter de ces deux variables.

---

### Évaluation sur le jeu d'entraînement et de test pour détecter le surapprentissage

Ci-dessous, on compare les performances sur l'ensemble d'entraînement et sur l'ensemble de test afin de vérifier si les modèles surapprennent.

In [ ]:
# Logistic Regression Evaluation
print("=== Logistic Regression Performance ===")

y_pred_train_logreg = logreg.predict(X_train)
y_proba_train_logreg = logreg.predict_proba(X_train)[:,1]
print("\n--- Training Set ---")
print("Accuracy:", accuracy_score(y_train, y_pred_train_logreg))
print("AUC:", roc_auc_score(y_train, y_proba_train_logreg))
print("Classification Report:\n", classification_report(y_train, y_pred_train_logreg))

print("\n--- Testing Set ---")
print("Accuracy:", accuracy_score(y_test, y_pred_logreg))
print("AUC:", roc_auc_score(y_test, y_proba_logreg))
print("Classification Report:\n", classification_report(y_test, y_pred_logreg))


# Random Forest Evaluation
print("\n\n=== Random Forest Performance ===")
y_pred_train_rf = rf.predict(X_train)
y_proba_train_rf = rf.predict_proba(X_train)[:,1]
print("\n--- Training Set ---")
print("Accuracy:", accuracy_score(y_train, y_pred_train_rf))
print("AUC:", roc_auc_score(y_train, y_proba_train_rf))
print("Classification Report:\n", classification_report(y_train, y_pred_train_rf))

print("\n--- Testing Set ---")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("AUC:", roc_auc_score(y_test, y_proba_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

In [ ]:
#TODO confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred_logreg)

print("Matrice de confusion:")
print(cm)


Le seuil `chosen_threshold` est celui qui permet une sensibilité ≥ 0.90. Vérifions la matrice de confusion avec ce seuil.

### Amélioration du Modèle de Régression Logistique

À ce stade, nous avons constaté que la Régression Logistique avait des difficultés à détecter la classe positive (infectés). Nous allons :

1. Appliquer un poids de classe (`class_weight='balanced'`) afin d'accorder plus d'importance aux individus infectés (minorité).
2. Effectuer une recherche d'hyperparamètres sur `C` (paramètre de régularisation) pour trouver une configuration plus adaptée.
3. Réévaluer les performances du modèle amélioré, en particulier la sensibilité (Recall) pour la classe 1.
4. Afficher la courbe ROC pour visualiser l'amélioration (ou non) du modèle.

In [ ]:
from sklearn.model_selection import GridSearchCV

# On reprend X_train, X_test, y_train, y_test du contexte précédent
# Assurez-vous que X_train, X_test, y_train, y_test sont déjà définis dans le notebook.

# Modèle de Régression Logistique avec class_weight='balanced'
logreg_balanced = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')

# Définition de la grille de paramètres pour C
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100]
}

# Utilisation de GridSearchCV pour trouver la meilleure valeur de C
grid_search = GridSearchCV(logreg_balanced, param_grid, scoring='roc_auc', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_logreg = grid_search.best_estimator_

print("Meilleurs paramètres trouvés :", grid_search.best_params_)
print("AUC sur l'ensemble d'entraînement (CV) :", grid_search.best_score_)

### Évaluation du modèle amélioré

Maintenant que nous avons un modèle de Régression Logistique avec class_weight et un C optimisé, évaluons ses performances sur le jeu de test. Nous prêterons attention à la matrice de confusion, au rapport de classification, et à la courbe ROC pour constater toute amélioration par rapport au modèle initial.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score

# Prédictions avec le meilleur modèle Logistic Regression
y_pred_best_logreg = best_logreg.predict(X_test)
y_proba_best_logreg = best_logreg.predict_proba(X_test)[:,1]

# Matrice de confusion et rapport de classification
cm_best = confusion_matrix(y_test, y_pred_best_logreg)
print("Matrice de confusion (Logistic Regression améliorée) :\n", cm_best)
print("\nRapport de classification (Logistic Regression améliorée) :")
print(classification_report(y_test, y_pred_best_logreg))

# Calcul de l'AUC
auc_best = roc_auc_score(y_test, y_proba_best_logreg)
print("AUC sur le jeu de test (Logistic Regression améliorée):", auc_best)

### Courbe ROC du Modèle Amélioré

Nous allons tracer la courbe ROC du nouveau modèle Logistic Regression amélioré et la comparer visuellement à la version initiale, si vous avez conservé ses valeurs. Cela permettra de voir si le modèle a gagné en capacité discriminante, en particulier pour la détection de la classe minoritaire.

In [ ]:
# Si vous avez conservé y_proba_logreg (probabilités de l'ancien modèle), vous pouvez le comparer.
# Sinon, on va simplement tracer la courbe du nouveau modèle amélioré.

fpr_best, tpr_best, _ = roc_curve(y_test, y_proba_best_logreg)

plt.figure(figsize=(8,6))

# Courbe du nouveau modèle
plt.plot(fpr_best, tpr_best, label=f"Logistic Regression Améliorée (AUC={auc_best:.2f})")

# Optionnel : si vous avez l'ancien y_proba_logreg, décommentez les deux lignes ci-dessous pour comparaison
# fpr_old, tpr_old, _ = roc_curve(y_test, y_proba_logreg)
# plt.plot(fpr_old, tpr_old, label=f"Logistic Regression Initiale (AUC={roc_auc_score(y_test, y_proba_logreg):.2f})", linestyle='--')

plt.plot([0,1],[0,1],'--',color='gray')
plt.xlabel("Taux de Faux Positifs (FPR)")
plt.ylabel("Taux de Vrais Positifs (TPR)")
plt.title("Courbe ROC - Logistic Regression Améliorée")
plt.legend()
plt.show()

### Analyse des Résultats

- En appliquant `class_weight='balanced'`, le modèle Logistic Regression accorde plus d'importance à la classe minoritaire. Couplé à un ajustement du paramètre `C`, cela devrait améliorer la capacité du modèle à détecter les individus infectés (augmenter le rappel pour la classe 1).
- L'AUC, la matrice de confusion, le rapport de classification et la courbe ROC permettent de constater cette amélioration.
- Si la sensibilité (Recall) de la classe 1 est désormais plus élevée et l'AUC plus grande que précédemment, le modèle est mieux adapté à l'objectif du projet.

### Nouvelle tentative d'amélioration du modèle Logistic Regression

Dans cette section, nous allons créer un nouveau modèle de régression logistique avec quelques améliorations afin d'obtenir de meilleurs résultats :

1. Utiliser `class_weight='balanced'` pour tenir compte du déséquilibre de classe.
2. Utiliser une recherche d'hyperparamètres (`GridSearchCV`) afin de trouver une valeur de `C` plus adaptée.

Nous espérons ainsi augmenter la capacité du modèle à détecter la classe minoritaire.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, ConfusionMatrixDisplay

# Assurez-vous que X_train, X_test, y_train, y_test sont déjà définis

# Création d'un pipeline avec un scaler et la régression logistique
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logistic', LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42))
])

# Grille de paramètres pour la recherche
param_grid = {
    'logistic__C': [0.01, 0.1, 1, 10, 100]
}

# Mise en place de la recherche par validation croisée
grid_search = GridSearchCV(pipeline, param_grid, scoring='roc_auc', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

improved_logistic_model = grid_search.best_estimator_

print("Meilleurs paramètres trouvés:", grid_search.best_params_)
print("Meilleure AUC en validation croisée:", grid_search.best_score_)

### Évaluation du nouveau modèle de Régression Logistique

Nous allons maintenant évaluer le modèle `improved_logistic_model` sur le jeu de test et comparer avec l'ancien modèle.

In [ ]:
# Prédictions sur le jeu de test
y_pred_improved = improved_logistic_model.predict(X_test)
y_proba_improved = improved_logistic_model.predict_proba(X_test)[:,1]

# Matrice de confusion
cm_improved = confusion_matrix(y_test, y_pred_improved)
print("Matrice de confusion (Logistic Regression améliorée) :\n", cm_improved)

disp = ConfusionMatrixDisplay(confusion_matrix=cm_improved)
disp.plot()
plt.show()

# Rapport de classification
print("\nRapport de classification (Logistic Regression améliorée) :")
print(classification_report(y_test, y_pred_improved))

# Calcul de l'AUC
auc_improved = roc_auc_score(y_test, y_proba_improved)
print("AUC sur le jeu de test (Logistic Regression améliorée):", auc_improved)

### Courbe ROC du Modèle Amélioré

Nous allons tracer la courbe ROC du nouveau modèle afin de visualiser l'amélioration potentielle par rapport à l'ancien modèle.

In [ ]:
fpr_improved, tpr_improved, _ = roc_curve(y_test, y_proba_improved)

plt.figure(figsize=(8,6))

plt.plot(fpr_improved, tpr_improved, label=f"Improved Logistic Regression (AUC={auc_improved:.2f})")

# Si vous avez conservé y_proba_logreg de l'ancien modèle, vous pouvez le comparer :
# fpr_old, tpr_old, _ = roc_curve(y_test, y_proba_logreg)
# old_auc = roc_auc_score(y_test, y_proba_logreg)
# plt.plot(fpr_old, tpr_old, label=f"Old Logistic Regression (AUC={old_auc:.2f})", linestyle='--')

plt.plot([0,1],[0,1],'--',color='gray')
plt.xlabel("Taux de Faux Positifs (FPR)")
plt.ylabel("Taux de Vrais Positifs (TPR)")
plt.title("Courbe ROC - Logistic Regression Améliorée")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Predict probabilities for both models
y_proba_rf = rf.predict_proba(X_test)[:,1]
y_proba_improved = improved_logistic_model.predict_proba(X_test)[:,1]

# Compute ROC curves and AUC
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_proba_rf)
auc_rf = roc_auc_score(y_test, y_proba_rf)

fpr_improved, tpr_improved, _ = roc_curve(y_test, y_proba_improved)
auc_improved = roc_auc_score(y_test, y_proba_improved)

# Plot ROC curves
plt.figure(figsize=(8,6))
plt.plot(fpr_rf, tpr_rf, label=f"Random Forest (AUC={auc_rf:.2f})")
plt.plot(fpr_improved, tpr_improved, label=f"Improved Logistic Regression (AUC={auc_improved:.2f})")
plt.plot([0,1],[0,1],'--',color='gray')
plt.xlabel("Taux de Faux Positifs (FPR)")
plt.ylabel("Taux de Vrais Positifs (TPR)")
plt.title("Courbes ROC - Comparaison Random Forest vs Logistic Regression Améliorée")
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# Load the data
data_new = pd.read_csv("projet_1_data.csv")
X_new = data_new[['x1', 'x2', 'x3', 'x4', 'x5']]
y_new = data_new['y']

# Split into training and test sets
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(
    X_new, y_new, test_size=0.2, random_state=42
)

# Ensure test data columns match training data columns
X_test_new = X_test_new[X_train_new.columns]

# Define the pipeline
new_pipeline = Pipeline([
    ('poly_features', PolynomialFeatures()),          # Polynomial expansion
    ('scaler_step', StandardScaler()),                # Scaling
    ('logistic_model', LogisticRegression(
        class_weight='balanced', max_iter=2000, random_state=42))
])

# Parameter grid for the pipeline
new_param_grid = {
    'poly_features__degree': [1, 2, 3],
    'logistic_model__C': [0.01, 0.1, 1, 10, 100]
}

# Set up GridSearchCV using DataFrames
new_grid_search = GridSearchCV(
    new_pipeline,
    new_param_grid,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    verbose=1
)
new_grid_search.fit(X_train_new, y_train_new)

# Best parameters and score
print("Best parameters from new_grid_search:")
print(new_grid_search.best_params_)
print(f"Best cross-validation AUC from new_grid_search: {new_grid_search.best_score_:.4f}")

# Evaluate on the test set using DataFrames
best_new_model = new_grid_search.best_estimator_

# Make predictions
y_pred_new = best_new_model.predict(X_test_new)
y_proba_new = best_new_model.predict_proba(X_test_new)[:, 1]

# Confusion matrix and classification report
new_cm = confusion_matrix(y_test_new, y_pred_new)
print("\nTest Set Confusion Matrix from best_new_model:")
print(new_cm)

print("\nClassification Report on Test Set from best_new_model:")
print(classification_report(y_test_new, y_pred_new))

# AUC score
new_auc_test = roc_auc_score(y_test_new, y_proba_new)
print(f"Test Set AUC from best_new_model: {new_auc_test:.4f}")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

# Predict probabilities for both models
y_proba_rf = rf.predict_proba(X_test_new)[:, 1]  # Probabilities from Random Forest
y_proba_new = best_new_model.predict_proba(X_test_new)[:, 1]  # Probabilities from Logistic Regression

# Compute ROC curves and AUC for Random Forest
fpr_rf, tpr_rf, _ = roc_curve(y_test_new, y_proba_rf)
auc_rf = roc_auc_score(y_test_new, y_proba_rf)

# Compute ROC curves and AUC for Logistic Regression
fpr_new, tpr_new, _ = roc_curve(y_test_new, y_proba_new)
auc_new = roc_auc_score(y_test_new, y_proba_new)

# Plot ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, label=f"Random Forest (AUC={auc_rf:.4f})")
plt.plot(fpr_new, tpr_new, label=f"Improved Logistic Regression (AUC={auc_new:.4f})")
plt.plot([0, 1], [0, 1], '--', color='gray')  # Diagonal line for random guessing

# Add labels, title, and legend
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve Comparison: Random Forest vs Logistic Regression")
plt.legend(loc="lower right")
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score

# Prédictions avec le meilleur modèle Logistic Regression (best_new_model)
y_pred_best_logreg = best_new_model.predict(X_test_new)
y_proba_best_logreg = best_new_model.predict_proba(X_test_new)[:, 1]

# Matrice de confusion et rapport de classification
cm_best = confusion_matrix(y_test_new, y_pred_best_logreg)
print("Matrice de confusion (Logistic Regression améliorée) :\n", cm_best)
print("\nRapport de classification (Logistic Regression améliorée) :")
print(classification_report(y_test_new, y_pred_best_logreg))

# Calcul de l'AUC
auc_best = roc_auc_score(y_test_new, y_proba_best_logreg)
print("AUC sur le jeu de test (Logistic Regression améliorée):", auc_best)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score

# Load the data
data_new = pd.read_csv("projet_1_data.csv")
X_new = data_new[['x1', 'x2', 'x3', 'x4', 'x5']]
y_new = data_new['y']

# Split into training and test sets
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(
    X_new, y_new, test_size=0.2, random_state=42
)

# Ensure test data columns match training data columns
X_test_new = X_test_new[X_train_new.columns]

# Define the pipeline
new_pipeline = Pipeline([
    ('poly_features', PolynomialFeatures()),          # Polynomial expansion
    ('scaler_step', StandardScaler()),                # Scaling
    ('logistic_model', LogisticRegression(
        class_weight='balanced', max_iter=2000, random_state=42))
])

# Parameter grid for the pipeline
new_param_grid = {
    'poly_features__degree': [1, 2, 3],
    'logistic_model__C': [0.01, 0.1, 1, 10, 100]
}

# Set up GridSearchCV using DataFrames
new_grid_search = GridSearchCV(
    new_pipeline,
    new_param_grid,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    verbose=1
)
new_grid_search.fit(X_train_new, y_train_new)

# Best parameters and score
print("Best parameters from new_grid_search:")
print(new_grid_search.best_params_)
print(f"Best cross-validation AUC from new_grid_search: {new_grid_search.best_score_:.4f}")

# Evaluate on the test set using DataFrames
best_new_model = new_grid_search.best_estimator_

# Make predictions
y_pred_best_new = best_new_model.predict(X_test_new)
y_proba_best_new = best_new_model.predict_proba(X_test_new)[:, 1]

# Confusion matrix and classification report
cm_best_new = confusion_matrix(y_test_new, y_pred_best_new)
print("\nTest Set Confusion Matrix from best_new_model:")
print(cm_best_new)

print("\nClassification Report on Test Set from best_new_model:")
print(classification_report(y_test_new, y_pred_best_new))

# AUC score
auc_best_new = roc_auc_score(y_test_new, y_proba_best_new)
print(f"Test Set AUC from best_new_model: {auc_best_new:.4f}")

# Predictions for rf model
y_pred_rf = rf.predict(X_test_new)
y_proba_rf = rf.predict_proba(X_test_new)[:, 1]

# Confusion matrix for rf model
cm_rf = confusion_matrix(y_test_new, y_pred_rf)
print("\nTest Set Confusion Matrix from rf model:")
print(cm_rf)

print("\nClassification Report on Test Set from rf model:")
print(classification_report(y_test_new, y_pred_rf))

# AUC score for rf model
auc_rf = roc_auc_score(y_test_new, y_proba_rf)
print(f"Test Set AUC from rf model: {auc_rf:.4f}")

# Function to compute rates and errors
def compute_rates(cm):
    TN, FP, FN, TP = cm.ravel()
    false_positive_rate = FP / (FP + TN)
    false_negative_rate = FN / (FN + TP)
    total_error_rate = (FP + FN) / (TN + FP + FN + TP)
    return false_positive_rate, false_negative_rate, total_error_rate

# Compute metrics for best_new_model
fpr_best_new, fnr_best_new, error_rate_best_new = compute_rates(cm_best_new)
print("Best New Model:")
print(f"False Positive Rate: {fpr_best_new:.4f}")
print(f"False Negative Rate: {fnr_best_new:.4f}")
print(f"Total Error Rate: {error_rate_best_new:.4f}")

# Compute metrics for rf model
fpr_rf, fnr_rf, error_rate_rf = compute_rates(cm_rf)
print("\nRandom Forest Model:")
print(f"False Positive Rate: {fpr_rf:.4f}")
print(f"False Negative Rate: {fnr_rf:.4f}")
print(f"Total Error Rate: {error_rate_rf:.4f}")

# Plot ROC curves
fpr_rf, tpr_rf, _ = roc_curve(y_test_new, y_proba_rf)
fpr_new, tpr_new, _ = roc_curve(y_test_new, y_proba_best_new)

plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, label=f"Random Forest (AUC={auc_rf:.4f})")
plt.plot(fpr_new, tpr_new, label=f"Improved Logistic Regression (AUC={auc_best_new:.4f})")
plt.plot([0, 1], [0, 1], '--', color='gray')  # Diagonal line for random guessing

# Add labels, title, and legend
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve Comparison: Random Forest vs Logistic Regression")
plt.legend(loc="lower right")
plt.grid()
plt.show()

In [ ]:

import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_curve,
    roc_auc_score,
    ConfusionMatrixDisplay,
)

# Load the dataset
data = pd.read_csv("projet_1_data.csv")
X = data[["x1", "x2", "x3", "x4", "x5"]]
y = data["y"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# Utility function to optimize the threshold for a specific False Negative Rate (FNR)
def optimize_threshold(y_true, y_proba, max_fnr=0.1):
    """
    Optimizes the decision threshold to ensure FNR ≤ max_fnr and minimizes FPR.

    Args:
    - y_true: Ground truth labels
    - y_proba: Predicted probabilities
    - max_fnr: Maximum allowable false negative rate

    Returns:
    - Best threshold meeting the FNR requirement with minimum FPR
    - FNR and FPR at the selected threshold
    """
    thresholds = np.linspace(0, 1, 101)
    best_threshold = 0.5
    best_fnr = 1.0
    best_fpr = 1.0

    for t in thresholds:
        y_pred = (y_proba >= t).astype(int)
        cm = confusion_matrix(y_true, y_pred)
        TN, FP, FN, TP = cm.ravel()

        fnr = FN / (FN + TP) if (FN + TP) > 0 else 1.0
        fpr = FP / (FP + TN) if (FP + TN) > 0 else 1.0

        # Ensure FNR ≤ max_fnr, and minimize FPR
        if fnr <= max_fnr and fpr < best_fpr:
            best_threshold = t
            best_fnr = fnr
            best_fpr = fpr

    return best_threshold, best_fnr, best_fpr


# --- Logistic Regression Model (Best New Model) ---
logistic_pipeline = Pipeline(
    [
        ("poly_features", PolynomialFeatures()),  # Polynomial expansion
        ("scaler", StandardScaler()),  # Scaling
        (
            "logistic_model",
            LogisticRegression(class_weight="balanced", random_state=42, max_iter=2000),
        ),
    ]
)
param_grid_lr = {
    "poly_features__degree": [1, 2, 3],
    "logistic_model__C": [0.01, 0.1, 1, 10, 100],
}

# Grid Search for Logistic Regression
grid_search_lr = GridSearchCV(
    logistic_pipeline, param_grid_lr, scoring="roc_auc", cv=5, n_jobs=-1
)
grid_search_lr.fit(X_train, y_train)

best_new_model = grid_search_lr.best_estimator_

# Predict probabilities for Logistic Regression
y_proba_lr = best_new_model.predict_proba(X_test)[:, 1]
lr_threshold, lr_fnr, lr_fpr = optimize_threshold(y_test, y_proba_lr, max_fnr=0.1)
y_pred_lr = (y_proba_lr >= lr_threshold).astype(int)

# --- Random Forest Model ---
random_forest_model = RandomForestClassifier(random_state=42)
param_grid_rf = {"n_estimators": [50, 100, 200], "max_depth": [3, 5, None]}

# Grid Search for Random Forest
grid_search_rf = GridSearchCV(
    random_forest_model, param_grid_rf, scoring="roc_auc", cv=5, n_jobs=-1
)
grid_search_rf.fit(X_train, y_train)

best_rf_model = grid_search_rf.best_estimator_

# Predict probabilities for Random Forest
y_proba_rf = best_rf_model.predict_proba(X_test)[:, 1]
rf_threshold, rf_fnr, rf_fpr = optimize_threshold(y_test, y_proba_rf, max_fnr=0.1)
y_pred_rf = (y_proba_rf >= rf_threshold).astype(int)

# --- Evaluation Metrics ---
def evaluate_model(name, y_true, y_pred, y_proba, threshold, fnr, fpr):
    print(f"=== {name} ===")
    print(f"Optimized Threshold: {threshold:.2f}")
    print(f"False Negative Rate (FNR): {fnr:.2f}")
    print(f"False Positive Rate (FPR): {fpr:.2f}\n")
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title(f"Confusion Matrix - {name}")
    plt.show()
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    print(f"AUC: {roc_auc_score(y_true, y_proba):.2f}")
    print("\n" + "-" * 50 + "\n")


# Evaluate Logistic Regression
evaluate_model(
    "Logistic Regression (Best New Model)",
    y_test,
    y_pred_lr,
    y_proba_lr,
    lr_threshold,
    lr_fnr,
    lr_fpr,
)


# Evaluate Random Forest
evaluate_model(
    "Random Forest",
    y_test,
    y_pred_rf,
    y_proba_rf,
    rf_threshold,
    rf_fnr,
    rf_fpr,
)

# --- ROC Curves ---
def plot_roc_curves(y_true, models):
    """
    Plots ROC curves for multiple models.

    Args:
    - y_true: Ground truth labels
    - models: List of tuples [(name, y_proba)], where y_proba are probabilities
    """
    plt.figure(figsize=(10, 8))
    for name, y_proba in models:
        fpr, tpr, _ = roc_curve(y_true, y_proba)
        auc = roc_auc_score(y_true, y_proba)
        plt.plot(fpr, tpr, label=f"{name} (AUC={auc:.2f})")
    plt.plot([0, 1], [0, 1], "--", color="gray")
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.title("ROC Curves")
    plt.legend()
    plt.grid()
    plt.show()


# Plot ROC curves
plot_roc_curves(
    y_test, [("Logistic Regression (Best New Model)", y_proba_lr), ("Random Forest", y_proba_rf)]
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# Extract reduced feature sets
X_train_reduced = X_train[['x1', 'x2', 'x3']]
X_test_reduced = X_test[['x1', 'x2', 'x3']]

# Train a new Random Forest model on only x1, x2, x3
rf_reduced = RandomForestClassifier(
    n_estimators=best_rf_model.n_estimators,
    max_depth=best_rf_model.max_depth,
    random_state=42
    # Add any other parameters that were used in best_rf_model if known
)
rf_reduced.fit(X_train_reduced, y_train)

# Evaluate the original best_rf_model on the test set
y_proba_full = best_rf_model.predict_proba(X_test)[:, 1]
y_pred_full = (y_proba_full >= 0.5).astype(int)
print("=== Best_RF_Model (Full Features) ===")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_full))
print("Classification Report:\n", classification_report(y_test, y_pred_full))
print("AUC:", roc_auc_score(y_test, y_proba_full))

# Evaluate the new reduced model on the test set
y_proba_reduced = rf_reduced.predict_proba(X_test_reduced)[:, 1]
y_pred_reduced = (y_proba_reduced >= 0.5).astype(int)
print("\n=== RF Model (x1, x2, x3 Only) ===")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_reduced))
print("Classification Report:\n", classification_report(y_test, y_pred_reduced))
print("AUC:", roc_auc_score(y_test, y_proba_reduced))

# Compare the AUC, precision, recall for the infected class, or other important metrics 
# to determine if performance is similar. If the results are close, it suggests you 
# may not need x4 and x5.

### Réduction du nombre de variables

Après analyse, nous avons déterminé que les variables `x1`, `x2`  sont suffisantes pour nos modèles. Nous allons donc nous concentrer uniquement sur ces trois variables pour la suite de notre analyse et modélisation.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# Extract reduced feature sets (only x1 and x2)
X_train_reduced_2 = X_train[['x1', 'x2']]
X_test_reduced_2 = X_test[['x1', 'x2']]

# Train a new Random Forest model on only x1, x2
rf_reduced_2 = RandomForestClassifier(
    n_estimators=best_rf_model.n_estimators,
    max_depth=best_rf_model.max_depth,
    random_state=42
    # If other hyperparameters were tuned, add them here to replicate best_rf_model's settings
)
rf_reduced_2.fit(X_train_reduced_2, y_train)

# Evaluate the original best_rf_model (full features)
y_proba_full = best_rf_model.predict_proba(X_test)[:, 1]
y_pred_full = (y_proba_full >= 0.5).astype(int)
print("=== Best_RF_Model (Full Features) ===")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_full))
print("Classification Report:\n", classification_report(y_test, y_pred_full))
print("AUC:", roc_auc_score(y_test, y_proba_full))

# Evaluate the model trained on only x1 and x2
y_proba_reduced_2 = rf_reduced_2.predict_proba(X_test_reduced_2)[:, 1]
y_pred_reduced_2 = (y_proba_reduced_2 >= 0.5).astype(int)
print("\n=== RF Model (x1, x2 Only) ===")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_reduced_2))
print("Classification Report:\n", classification_report(y_test, y_pred_reduced_2))
print("AUC:", roc_auc_score(y_test, y_proba_reduced_2))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# Load data
data = pd.read_csv("projet_1_data.csv")
X = data[['x1', 'x2']]  # Only x1 and x2
y = data['y']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Define the pipeline with only x1 and x2
pipeline_reduced = Pipeline([
    ('poly_features', PolynomialFeatures()),         # Polynomial features
    ('scaler', StandardScaler()),                    # Standard scaling
    ('logistic', LogisticRegression(
        class_weight='balanced', max_iter=2000, random_state=42))
])

# Parameter grid for the reduced feature set
param_grid_reduced = {
    'poly_features__degree': [1, 2, 3],
    'logistic__C': [0.01, 0.1, 1, 10, 100]
}

# GridSearchCV on the reduced feature set
grid_search_reduced = GridSearchCV(
    pipeline_reduced,
    param_grid_reduced,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1
)
grid_search_reduced.fit(X_train, y_train)

# The best model trained on only x1 and x2
best_new_model_reduced = grid_search_reduced.best_estimator_

# Evaluate on test set
y_pred_reduced = best_new_model_reduced.predict(X_test)
y_proba_reduced = best_new_model_reduced.predict_proba(X_test)[:, 1]

cm_reduced = confusion_matrix(y_test, y_pred_reduced)
print("Confusion Matrix (x1, x2 only):\n", cm_reduced)
print("\nClassification Report (x1, x2 only):")
print(classification_report(y_test, y_pred_reduced))
print("Test Set AUC (x1, x2 only):", roc_auc_score(y_test, y_proba_reduced))

### Réduction du nombre de variables

Après analyse, nous avons déterminé que les variables `x1`et `x2`  sont suffisantes pour nos modèles. Nous allons donc nous concentrer uniquement sur ces trois variables pour la suite de notre analyse et modélisation.

Après avoir comparé les performances du modèle de forêt aléatoire sur plusieurs jeux de variables, les résultats indiquent clairement que seules les variables x1 et x2 sont nécessaires pour obtenir de bonnes performances prédictives. En effet :
	•	Le modèle utilisant uniquement x1 et x2 atteint une sensibilité (rappel) plus élevée pour la détection des individus infectés que la version avec l’ensemble complet de variables (x1, x2, x3, x4, x5).
	•	L’AUC (Area Under the ROC Curve) et l’exactitude (accuracy) se maintiennent, voire s’améliorent, lorsque l’on retire les variables x3, x4 et x5.
	•	L’absence de dégradation des performances, voire leur amélioration, valide la pertinence d’abandonner ces trois variables supplémentaires.

En conclusion, les résultats démontrent que le modèle ne nécessite que x1 et x2 pour atteindre un niveau de performance équivalent, voire supérieur, par rapport à l’utilisation de toutes les variables. L’abandon de x3, x4 et x5 est donc pleinement justifié.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Predict probabilities for both models
y_pred_prob_rf = rf_reduced_2.predict_proba(X_test)[:, 1]
y_pred_prob_best = best_new_model_reduced.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC for rf_reduced_2
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_prob_rf)
auc_rf = auc(fpr_rf, tpr_rf)

# Compute ROC curve and AUC for best_new_model_reduced
fpr_best, tpr_best, _ = roc_curve(y_test, y_pred_prob_best)
auc_best = auc(fpr_best, tpr_best)

# Plot ROC curves
plt.figure()
plt.plot(fpr_rf, tpr_rf, label='rf_reduced_2 (AUC = %0.2f)' % auc_rf)
plt.plot(fpr_best, tpr_best, label='best_new_model_reduced (AUC = %0.2f)' % auc_best)
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc='lower right')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

# Predict classes for both models
y_pred_rf = rf_reduced_2.predict(X_test)
y_pred_best = best_new_model_reduced.predict(X_test)

# Compute confusion matrix for rf_reduced_2
tn_rf, fp_rf, fn_rf, tp_rf = confusion_matrix(y_test, y_pred_rf).ravel()

# Calculate false positive rate and false negative rate for rf_reduced_2
false_positive_rate_rf = fp_rf / (fp_rf + tn_rf)
false_negative_rate_rf = fn_rf / (fn_rf + tp_rf)

# Compute confusion matrix for best_new_model_reduced
tn_best, fp_best, fn_best, tp_best = confusion_matrix(y_test, y_pred_best).ravel()

# Calculate false positive rate and false negative rate for best_new_model_reduced
false_positive_rate_best = fp_best / (fp_best + tn_best)
false_negative_rate_best = fn_best / (fn_best + tp_best)

# Print the results
print("rf_reduced_2:")
print(f"False Positive Rate: {false_positive_rate_rf:.2f}")
print(f"False Negative Rate: {false_negative_rate_rf:.2f}\n")

print("best_new_model_reduced:")
print(f"False Positive Rate: {false_positive_rate_best:.2f}")
print(f"False Negative Rate: {false_negative_rate_best:.2f}")

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

def optimize_threshold(y_true, y_proba, max_fnr=0.1):
    """
    Finds a decision threshold that ensures FNR ≤ max_fnr.
    Among those that satisfy this condition, it picks the threshold with the lowest FPR.
    """
    thresholds = np.linspace(0, 1, 101)
    best_threshold = 0.5
    best_fnr = 1.0
    best_fpr = 1.0

    for t in thresholds:
        y_pred = (y_proba >= t).astype(int)
        cm = confusion_matrix(y_true, y_pred)
        TN, FP, FN, TP = cm.ravel()

        fnr = FN / (FN + TP) if (FN + TP) > 0 else 1.0
        fpr = FP / (FP + TN) if (FP + TN) > 0 else 1.0

        # Among thresholds that achieve FNR ≤ max_fnr, pick the one with the lowest FPR
        if fnr <= max_fnr and fpr < best_fpr:
            best_threshold = t
            best_fnr = fnr
            best_fpr = fpr

    return best_threshold, best_fnr, best_fpr

# Adjust threshold for the Random Forest model (using x1, x2)
y_proba_rf = rf_reduced_2.predict_proba(X_test[['x1','x2']])[:, 1]
rf_threshold, rf_fnr, rf_fpr = optimize_threshold(y_test, y_proba_rf, max_fnr=0.1)
y_pred_rf = (y_proba_rf >= rf_threshold).astype(int)
cm_rf = confusion_matrix(y_test, y_pred_rf)
TN_rf, FP_rf, FN_rf, TP_rf = cm_rf.ravel()

print("=== Random Forest (x1, x2) with Adjusted Threshold ===")
print("Selected Threshold:", rf_threshold)
print("Confusion Matrix:\n", cm_rf)
print("False Negative Rate (FNR): {:.2f}%".format(rf_fnr * 100))
print("False Positive Rate (FPR): {:.2f}%".format(rf_fpr * 100))
print("AUC:", roc_auc_score(y_test, y_proba_rf))
print("FNR ≤ 10%: ", rf_fnr <= 0.1)
print()

# Adjust threshold for the Logistic Regression model (using x1, x2)
y_proba_lr = best_new_model_reduced.predict_proba(X_test[['x1','x2']])[:, 1]
lr_threshold, lr_fnr, lr_fpr = optimize_threshold(y_test, y_proba_lr, max_fnr=0.1)
y_pred_lr = (y_proba_lr >= lr_threshold).astype(int)
cm_lr = confusion_matrix(y_test, y_pred_lr)
TN_lr, FP_lr, FN_lr, TP_lr = cm_lr.ravel()

print("=== Logistic Regression (x1, x2) with Adjusted Threshold ===")
print("Selected Threshold:", lr_threshold)
print("Confusion Matrix:\n", cm_lr)
print("False Negative Rate (FNR): {:.2f}%".format(lr_fnr * 100))
print("False Positive Rate (FPR): {:.2f}%".format(lr_fpr * 100))
print("AUC:", roc_auc_score(y_test, y_proba_lr))
print("FNR ≤ 10%: ", lr_fnr <= 0.1)
print()

---
#### Verification

Veuillez valider vos modèles à l'aide de la fonction ci-dessous. Le premier modèle doit être celui que vous considérez comme le plus performant.

⚠️ - Cette fonction vérifie uniquement que votre soumission est vérifiable. Elle ne donne aucune indication sur la validité de la soumission (e.g. type de ML) ou sa qualité.

Vous pouvez spécifier les caractéristiques principales sous deux formats :

- Une liste de caractéristiques commune à tous les modèles.
- Une liste de listes de caractéristiques, correspondant à chaque modèle (voir exemple), dans le même ordre.

Chaque list de caractéristiques peut être spécifiée de deux manières :

- Une liste de noms de colonnes pertinentes, dans l'ordre désiré, par exemple ['x3', 'x1', 'x4'].
- Une liste de valeurs booléennes correspondant aux colonnes, où True indique une colonne pertinente, par exemple [True, False, True, False, True] pour sélectionner les colonnes ['x1', 'x3', 'x5'].

Notez que si le nombre de caractéristiques attendues par votre modèle ne correspond pas au nombre de caractéristiques spécifiées dans la fonction, celle-ci ignorera cette liste de caractéristiques et supposera que le modèle inclut une étape de filtrage automatique des caractéristiques et validera le modèle avec toutes les variables du fichier de hold-out (dans ce cas la liste de caractéristiques sert uniquement à vérifier votre sélection).

Vous pouvez également définir un seuil de détection (e.g. pour contrôler le taux de fausses négatives): soit un seuil commun pour les deux modèles, soit une liste de seuils distincts par modèle (ou seuil=None pour utiliser les valeurs par défaut).

In [35]:
# executer la commande ci-dessous si nécessaire (enlever le #).
# !pip install -e ./eng209
from importlib import reload
import eng209.verify
reload(eng209.verify)
from eng209.verify import verify_q1


In [ ]:
# Remplacez best_model et second_best_model par vos modèles, et modifiez les caractéristiques et le seuil en conséquence.
best_model= rf_reduced_2
second_best_model=best_new_model_reduced

caracteristiques=['x1','x2']
seuil=[rf_threshold,lr_threshold]

# Verify models
verify_q1(best_model, second_best_model, caracteristiques=caracteristiques, seuil=seuil)
